In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import h2o
from h2o.automl import H2OAutoML
h2o.init(nthreads=7,
        max_mem_size='6g')

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,1 hour 33 mins
H2O cluster timezone:,Europe/Paris
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.0.3
H2O cluster version age:,25 days
H2O cluster name:,H2O_from_python_edu_yuaa4k
H2O cluster total nodes:,1
H2O cluster free memory:,386.7 Mb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"locked, healthy"


In [2]:
runtime = 300

In [3]:
train_high = h2o.import_file('../cleaned-data/08-train_high-colorAsInt-restAsDummies.csv')
train_low = h2o.import_file('../cleaned-data/08-train_low-colorAsInt-restAsDummies.csv')
test_high = h2o.import_file('../cleaned-data/08-test_high-colorAsInt-restAsDummies.csv')
test_low = h2o.import_file('../cleaned-data/08-test_low-colorAsInt-restAsDummies.csv')      
display(train_high)
test_high

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


id,carat,color,depth,table,x,y,z,cut_Fair,cut_Good,cut_Ideal,cut_Premium,cut_Very Good,clarity_I1,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2,price
8,2.05,7,62.4,57,8.17,8.09,5.08,0,0,1,0,0,0,0,1,0,0,0,0,0,18193
34,2.01,6,64.9,60,7.82,7.78,5.06,1,0,0,0,0,0,0,0,0,0,1,0,0,13744
42,2,5,64.2,52,7.91,7.88,5.07,0,1,0,0,0,0,0,0,1,0,0,0,0,11400
72,2,5,60.3,56,8.08,8.11,4.88,0,1,0,0,0,0,0,0,1,0,0,0,0,16425
82,2.28,3,62.7,57,8.43,8.36,5.26,0,0,1,0,0,0,0,0,0,0,1,0,0,16450
93,2.02,1,60.4,58,8.2,8.16,4.94,0,0,0,0,1,0,0,1,0,0,0,0,0,17357
141,2,1,60.7,60,8.07,8.11,4.91,0,0,0,1,0,0,0,0,1,0,0,0,0,15312
154,2.01,5,61.5,62,8.07,7.94,4.92,0,0,0,1,0,0,0,0,0,1,0,0,0,15475
165,2.31,6,62.9,58,8.43,8.34,5.27,0,0,0,1,0,0,0,0,1,0,0,0,0,17218
167,2.01,5,60.6,57,8.19,8.09,4.92,0,0,1,0,0,0,0,1,0,0,0,0,0,17383


id,carat,color,depth,table,x,y,z,cut_Fair,cut_Good,cut_Ideal,cut_Premium,cut_Very Good,clarity_I1,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
2,2.03,4,61.9,59,8.14,8.09,5.02,0,0,0,1,0,0,0,1,0,0,0,0,0
29,2.52,4,63.7,59,8.57,8.52,5.44,0,1,0,0,0,0,0,0,1,0,0,0,0
38,2,5,58.8,61,8.29,8.23,4.86,0,0,0,1,0,0,0,1,0,0,0,0,0
75,2.01,4,60.3,59,8.23,8.15,4.94,0,0,0,1,0,0,0,1,0,0,0,0,0
76,2.15,2,62.5,57,8.2,8.16,5.11,0,0,1,0,0,0,0,1,0,0,0,0,0
90,2.02,2,61.7,57,8.12,8.05,4.99,0,0,1,0,0,0,0,0,1,0,0,0,0
116,2.02,4,61.8,58,8.16,8.1,4.94,0,0,0,1,0,0,0,1,0,0,0,0,0
143,2.11,5,63.5,55,8.08,8.13,5.15,0,0,0,0,1,0,0,0,1,0,0,0,0
189,2.05,1,62.5,60,8.09,8.14,5.07,0,0,0,1,0,0,0,0,0,1,0,0,0
242,2.54,3,63.5,56,8.68,8.65,5.5,0,0,0,0,1,0,0,0,1,0,0,0,0


In [4]:
list(map(lambda x: print(x.shape),[train_high, train_low, test_high, test_low]))

(1615, 22)
(38730, 22)
(521, 21)
(12928, 21)


[None, None, None, None]

In [5]:
xTrainCols = train_high.col_names[1:-1]
xTrainCols

['carat',
 'color',
 'depth',
 'table',
 'x',
 'y',
 'z',
 'cut_Fair',
 'cut_Good',
 'cut_Ideal',
 'cut_Premium',
 'cut_Very Good',
 'clarity_I1',
 'clarity_IF',
 'clarity_SI1',
 'clarity_SI2',
 'clarity_VS1',
 'clarity_VS2',
 'clarity_VVS1',
 'clarity_VVS2']

In [6]:
train_frame = train_high
model_name = 'diamonds_high'
runtime = 180

aml_high = H2OAutoML(max_runtime_secs = runtime, project_name = f'{model_name}', keep_cross_validation_models=True)
aml_high.train(x=xTrainCols, y = 'price', training_frame = train_frame)
pred_high = aml_high.leader.predict(test_high)
display(pred_high)
display(aml_high.leaderboard)

AutoML progress: |
11:28:28.614: New models will be added to existing leaderboard diamonds_high@@price (leaderboard frame=null) with already 54 models.

████████████████████████████████████Failed polling AutoML progress log: HTTP 500 Server Error:
'<html>\n<head>\n<meta http-equiv="Content-Type" content="text/html;charset=ISO-8859-1"/>\n<title>Error 500 Server Error</title>\n</head>\n<body>\n<h2>HTTP ERROR: 500</h2>\n<p>Problem accessing /99/AutoML/diamonds_high@@price. Reason:\n<pre>    Server Error</pre></p>\n\n                                                \n                                                \n                                                \n                                                \n                                                \n                                                \n                                                \n                                                \n                                                \n                               

/home/edu/.local/lib/python3.7/site-packages/h2o/job.py:70: UserWarning: Test/Validation dataset is missing column 'cut_0': substituting in a column of NaN
  warnings.warn(w)
/home/edu/.local/lib/python3.7/site-packages/h2o/job.py:70: UserWarning: Test/Validation dataset is missing column 'cut_1': substituting in a column of NaN
  warnings.warn(w)
/home/edu/.local/lib/python3.7/site-packages/h2o/job.py:70: UserWarning: Test/Validation dataset is missing column 'cut_2': substituting in a column of NaN
  warnings.warn(w)
/home/edu/.local/lib/python3.7/site-packages/h2o/job.py:70: UserWarning: Test/Validation dataset is missing column 'cut_3': substituting in a column of NaN
  warnings.warn(w)
/home/edu/.local/lib/python3.7/site-packages/h2o/job.py:70: UserWarning: Test/Validation dataset is missing column 'cut_4': substituting in a column of NaN
  warnings.warn(w)
/home/edu/.local/lib/python3.7/site-packages/h2o/job.py:70: UserWarning: Test/Validation dataset is missing column 'color_1':

predict
16258.6
17287.5
15744.8
16308.7
16393.5
15852.7
16483.2
15855.5
16771.3
17200


model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_BestOfFamily_AutoML_20200302_095451,2.77492e+06,1665.81,2.77492e+06,1295.86,0.118979
StackedEnsemble_AllModels_AutoML_20200302_095451,2.7786e+06,1666.91,2.7786e+06,1295.38,0.119127
XGBoost_1_AutoML_20200302_095451,2.80204e+06,1673.93,2.80204e+06,1319.05,0.120058
XGBoost_grid__1_AutoML_20200302_095451_model_2,2.83379e+06,1683.39,2.83379e+06,1326.08,0.120946
XGBoost_grid__1_AutoML_20200302_095451_model_3,2.83793e+06,1684.62,2.83793e+06,1326.07,0.121122
XGBoost_2_AutoML_20200302_095451,2.8446e+06,1686.59,2.8446e+06,1322.08,0.120807
XGBoost_3_AutoML_20200302_095451,2.85922e+06,1690.92,2.85922e+06,1339.77,0.121469
GBM_4_AutoML_20200302_095451,2.85974e+06,1691.08,2.85974e+06,1323.8,0.121413
XGBoost_grid__1_AutoML_20200302_095451_model_4,2.87104e+06,1694.41,2.87104e+06,1317.51,0.121331
GBM_2_AutoML_20200302_095451,2.87277e+06,1694.92,2.87277e+06,1323.63,0.122075


In [7]:
tableHighPreds = test_high['id'].cbind(pred_high)
tableHighPreds = tableHighPreds.rename(columns={'predict':'price'})

In [8]:
train_frame = train_low
model_name = 'diamonds_low'

aml_low = H2OAutoML(max_runtime_secs = runtime, project_name = f'{model_name}', keep_cross_validation_models=True)
aml_low.train(x=xTrainCols, y = 'price', training_frame = train_frame)
pred_low = aml_low.leader.predict(test_low)
display(pred_low)
display(aml_low.leaderboard)

AutoML progress: |
11:31:30.718: New models will be added to existing leaderboard diamonds_low@@price (leaderboard frame=null) with already 10 models.

████████████████████████████████████████████████████████| 100%

11:34:32.241: StackedEnsemble_BestOfFamily_AutoML_20200302_113130 [StackedEnsemble best (built using top model from each algorithm type)] failed: water.exceptions.H2OIllegalArgumentException: Failed to find the xval predictions frame. . .  Looks like keep_cross_validation_predictions wasn't set when building the models, or the frame was deleted.
11:34:33.248: StackedEnsemble_AllModels_AutoML_20200302_113130 [StackedEnsemble all (built using all AutoML models)] failed: water.exceptions.H2OIllegalArgumentException: Failed to find the xval predictions frame. . .  Looks like keep_cross_validation_predictions wasn't set when building the models, or the frame was deleted.

stackedensemble prediction progress: |████████████████████████████████████| 100%


predict
6582.53
1644.72
8037.98
1773.58
2706.57
1866.29
1706.03
4116.48
9271.52
3170.92


model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_AllModels_AutoML_20200302_095957,183477,428.342,183477,225.514,0.0900198
StackedEnsemble_BestOfFamily_AutoML_20200302_095957,184253,429.247,184253,227.276,0.0916806
XGBoost_1_AutoML_20200302_095957,185430,430.616,185430,229.266,0.0933194
XGBoost_2_AutoML_20200302_095957,192142,438.34,192142,227.848,0.0885417
DRF_1_AutoML_20200302_095957,215090,463.778,215090,245.752,0.0980872
XGBoost_3_AutoML_20200302_095957,237756,487.603,237756,271.153,0.119445
GBM_1_AutoML_20200302_095957,402840,634.697,402840,370.03,0.190534
XGBoost_1_AutoML_20200302_113130,792971,890.489,792971,466.907,0.158628
XGBoost_2_AutoML_20200302_113130,802200,895.656,802200,468.001,0.156882
XGBoost_3_AutoML_20200302_113130,812149,901.193,812149,485.134,0.172407


In [9]:
tableLowPreds = test_low['id'].cbind(pred_low)
tableLowPreds = tableLowPreds.rename(columns={'predict':'price'})

In [10]:
combinedTable = tableLowPreds.concat(tableHighPreds, axis=0)
combinedTable.shape

(13449, 2)

In [11]:
h2o.export_file(frame=combinedTable,path='../predictions/15-Auto-ML-Split-data-colorAsInt-restAsDummies-v4.csv')

H2OResponseError: Server error water.exceptions.H2OIllegalArgumentException:
  Error: Illegal argument: ../predictions/13-Auto-ML-Split-data-colorAsInt-restAsDummies-v4.csv of function: exportFrame: File ../predictions/13-Auto-ML-Split-data-colorAsInt-restAsDummies-v4.csv already exists!
  Request: POST /3/Frames/py_511_sid_9049/export
    data: {'path': '../predictions/13-Auto-ML-Split-data-colorAsInt-restAsDummies-v4.csv', 'num_parts': '1', 'force': 'False', 'separator': '44'}
